In [5]:
import json
import networkx as nx
import numpy as np

data_root='../../dropbox/data/ind_ppi/'

from networkx.readwrite import json_graph
G_data = json.load(open(data_root + "/ppi-G.json"))
G = json_graph.node_link_graph(G_data)

feats = np.load('%s/ppi-feats.npy' % data_root)
labels = json.load(open(data_root + "/ppi-class_map.json"))
self_loop = 1

In [6]:
import os
out_root='../../dropbox/data/ppi-self-%d' % self_loop
if not os.path.isdir(out_root):
    os.mkdir(out_root)


In [56]:
with open('%s/adj_list.txt' % out_root, 'w') as f:
    for i in range(len(G)):
        edges = set(G.neighbors(i))
        assert len(G.neighbors(i)) == len(edges)
        if i in edges:
            if not self_loop:
                edges.remove(i)
        f.write('%d' % len(edges))
        for e in edges:
            f.write(' %d' % e)
        f.write('\n')

In [7]:
with open('%s/meta.txt' % out_root, 'w') as f:
    f.write('%d %d %d\n' % (len(G), len(labels['0']), feats.shape[1]))
    
# with open('%s/features.txt' % out_root, 'w') as f:
#     for i in range(feats.shape[0]):
#         feat = []
#         for j in range(feats.shape[1]):
#             if feats[i, j] > 0:
#                 feat.append('%d:%.2f' % (j, feats[i, j]))
#         f.write('%d' % len(feat))
#         for w in feat:
#             f.write(' %s' % w)
#         f.write('\n')
with open('%s/features.txt' % out_root, 'w') as f:
    for i in range(feats.shape[0]):
        for j in range(feats.shape[1]):
            if j:
                f.write(' ')
            f.write('%.2f' % feats[i, j])
        f.write('\n')

In [58]:
with open('%s/label.txt' % out_root, 'w') as f:
    for i in range(len(G)):
        l = labels[str(i)]
        for j in range(len(l)):
            if j:
                f.write(' ')
            f.write('%d' % l[j])
        f.write('\n')

In [59]:
train_list = []
val_list = []
test_list = []

for i in range(len(G)):
    if G.node[i]['test']:
        test_list.append(i)
    elif G.node[i]['val']:
        val_list.append(i)
    else:
        train_list.append(i)

In [60]:
with open('%s/train_idx.txt' % out_root, 'w') as f:
    for i in train_list:
        f.write('%d\n' % i)
with open('%s/val_idx.txt' % out_root, 'w') as f:
    for i in val_list:
        f.write('%d\n' % i)
with open('%s/test_idx.txt' % out_root, 'w') as f:
    for i in test_list:
        f.write('%d\n' % i)


In [24]:
id_map = json.load(open(data_root + "/ppi-id_map.json"))

In [28]:
class_map = json.load(open(data_root + "/ppi-class_map.json"))

In [34]:
type(class_map['0'])

list

In [7]:
train_list = []
broken_count= 0
for node in G.nodes():
    if not 'val' in G.node[node] or not 'test' in G.node[node]:
        G.remove_node(node)
        broken_count += 1
    if not G.node[node]['val'] and not G.node[node]['test']:
        train_list.append(node)
print(broken_count)
print(max(train_list), min(train_list), len(train_list))

0
44905 0 44906
